In [8]:
# SEL 1: Instalasi Library
# Jalankan sel ini terlebih dahulu
!pip install streamlit pyngrok joblib scikit-learn tensorflow -q

In [14]:
# SEL 2: Menulis File app.py
# Pastikan '%%writefile app.py' adalah baris PERTAMA di sel ini
%%writefile app.py

import streamlit as st
import joblib
import pandas as pd
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import os

# =============================================================================
# KONFIGURASI PENTING
# =============================================================================

CLASS_NAMES = ['Glioma', 'Meningioma', 'No Tumor', 'Pituitary']

IMG_HEIGHT = 224
IMG_WIDTH = 224

RF_MODEL_PATH = r'/content/random_forest_model.pkl'
XGB_MODEL_PATH = r'/content/xgboost_model.pkl'

# =============================================================================
# Fungsi Helper
# =============================================================================

@st.cache_resource
def load_all_models():
    try:
        rf_model = joblib.load(RF_MODEL_PATH)
    except Exception as e:
        st.error(f"Error RF: {e}")
        rf_model = None

    try:
        xgb_model = joblib.load(XGB_MODEL_PATH)
    except Exception as e:
        st.error(f"Error XGB: {e}")
        xgb_model = None

    base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')
    base_model.trainable = False

    return rf_model, xgb_model, base_model

def extract_features(img, base_model):
    if img.mode != 'RGB':
        img = img.convert('RGB')

    img_resized = img.resize((IMG_HEIGHT, IMG_WIDTH))
    img_array = keras_image.img_to_array(img_resized)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    img_preprocessed = preprocess_input(img_array_expanded)
    features = base_model.predict(img_preprocessed)
    return features

# =============================================================================
# Layout Aplikasi
# =============================================================================

st.set_page_config(page_title="Analisis Tumor Otak", layout="wide")
st.title("🧠 Analisis Citra MRI untuk Deteksi Tumor Otak")
st.markdown("Perbandingan Detail Probabilitas: **Random Forest** vs **XGBoost**")

rf_model, xgb_model, feature_extractor = load_all_models()
models_loaded = rf_model is not None and xgb_model is not None

if models_loaded:
    st.success("Sistem Siap: Semua model berhasil dimuat.")
else:
    st.error("Gagal memuat model. Cek file .pkl Anda.")

st.markdown("---")

uploaded_file = st.file_uploader("Unggah Citra MRI", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    col_img, col_btn = st.columns([1, 2])
    with col_img:
        image = Image.open(uploaded_file)
        st.image(image, caption="Citra Input", width=250)

    with col_btn:
        st.write("Klik tombol di bawah untuk melihat hasil analisis detail.")
        classify_button = st.button("🔍 Mulai Analisis Mendalam", disabled=not models_loaded)

    if classify_button and models_loaded:
        with st.spinner("Mengekstrak fitur MobileNetV2 dan menghitung probabilitas..."):

            # 1. Ekstraksi Fitur
            features = extract_features(image, feature_extractor)

            # --- RANDOM FOREST ---
            try:
                rf_probs = rf_model.predict_proba(features)[0]
                rf_conf = np.max(rf_probs)
                rf_label = CLASS_NAMES[np.argmax(rf_probs)]

                df_rf = pd.DataFrame({
                    "Jenis Tumor": CLASS_NAMES,
                    "Probabilitas": rf_probs
                })
            except Exception as e:
                st.error(f"RF Error: {e}")
                rf_label, rf_conf = "Error", 0
                df_rf = pd.DataFrame()

            # --- XGBOOST ---
            try:
                xgb_probs = xgb_model.predict_proba(features)[0]
                xgb_conf = np.max(xgb_probs)
                xgb_label = CLASS_NAMES[np.argmax(xgb_probs)]

                df_xgb = pd.DataFrame({
                    "Jenis Tumor": CLASS_NAMES,
                    "Probabilitas": xgb_probs
                })
            except Exception as e:
                st.error(f"XGB Error: {e}")
                xgb_label, xgb_conf = "Error", 0
                df_xgb = pd.DataFrame()

            st.markdown("---")
            st.subheader("📊 Hasil Analisis & Perbandingan Probabilitas")

            col1, col2 = st.columns(2)

            # === KOLOM RANDOM FOREST ===
            with col1:
                st.info(f"**Random Forest** memprediksi: **{rf_label}**")
                st.metric(label="Confidence Tertinggi", value=f"{rf_conf*100:.2f}%")

                st.markdown("###### Detail Probabilitas RF:")

                # KEMBALI KE VERSI AWAL (NATIVE STREAMLIT)
                st.dataframe(
                    df_rf,
                    column_config={
                        "Probabilitas": st.column_config.ProgressColumn(
                            "Confidence",
                            format="%.2f%%",
                            min_value=0,
                            max_value=1,
                        ),
                    },
                    hide_index=True,
                    use_container_width=True
                )

            # === KOLOM XGBOOST ===
            with col2:
                st.success(f"**XGBoost** memprediksi: **{xgb_label}**")
                st.metric(label="Confidence Tertinggi", value=f"{xgb_conf*100:.2f}%")

                st.markdown("###### Detail Probabilitas XGBoost:")

                # KEMBALI KE VERSI AWAL (NATIVE STREAMLIT)
                st.dataframe(
                    df_xgb,
                    column_config={
                        "Probabilitas": st.column_config.ProgressColumn(
                            "Confidence",
                            format="%.2f%%",
                            min_value=0,
                            max_value=1,
                        ),
                    },
                    hide_index=True,
                    use_container_width=True
                )

            # === INSIGHT TAMBAHAN ===
            sorted_probs = np.sort(xgb_probs)[::-1]
            gap = sorted_probs[0] - sorted_probs[1]

            if gap < 0.15:
                st.warning(f"⚠️ **Catatan Analisis:** Model XGBoost terlihat agak ragu (selisih prediksi ke-1 dan ke-2 hanya {gap*100:.1f}%). Disarankan validasi ahli.")

else:
    if not uploaded_file:
        st.info("Upload gambar untuk melihat prediksi.")

Overwriting app.py


In [15]:
# SEL 3: Menjalankan Streamlit dengan Ngrok
# Ganti <TOKEN_AUTH_NGROK_ANDA> dengan token Anda
from pyngrok import ngrok
import os

# Masukkan Authtoken Ngrok Anda di sini
AUTHTOKEN = "2zqw3dtSEzhJGnNXOmBPnPHbdaZ_2yZ811k2wM3fdHx1WKx3W"  # <-- GANTI DENGAN TOKEN ANDA
os.system(f"ngrok config add-authtoken {AUTHTOKEN}")

# Buka tunnel ke port 8501 (port default Streamlit)
public_url = ngrok.connect(8501)
print(f"URL Publik Streamlit Anda: {public_url}")

# Jalankan aplikasi Streamlit
!streamlit run app.py

URL Publik Streamlit Anda: NgrokTunnel: "https://677b70084c3a.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.80.79.5:8501

2025-11-19 15:34:07.827847: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763566447.869881   11513 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763566447.883005   11513 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763566447.917142   11513 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target 

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
